### Load ENV and Model

In [7]:
from dotenv import load_dotenv
load_dotenv()
from langchain_groq import ChatGroq
model = ChatGroq(model="llama-3.1-8b-instant")

### Load the Documents

In [9]:
from langchain_community.document_loaders import WebBaseLoader

urls = [
    "https://www.geeksforgeeks.org/artificial-intelligence/introduction-to-langchain/",
    "https://www.ibm.com/think/topics/langchain",
    "https://www.digitalocean.com/community/conceptual-articles/langchain-framework-explained"
]

loader = WebBaseLoader(web_paths=urls)
final_loader = loader.load()


### Splitters

In [10]:
from langchain_text_splitters import RecursiveCharacterTextSplitter

splitter = RecursiveCharacterTextSplitter(chunk_size=500,chunk_overlap=70)
final_split = splitter.split_documents(final_loader)


### Embeddings the Documents

In [11]:
from langchain_huggingface import HuggingFaceEmbeddings
embeddings = HuggingFaceEmbeddings(model_name="all-MiniLM-L6-v2")
texts = [doc.page_content for doc in final_split]
vector = embeddings.embed_documents(texts)


### Store in Vector DB

In [12]:
from langchain_community.vectorstores import FAISS

store = FAISS.from_documents(documents=final_split,embedding=embeddings)

### Create a Retriever

In [13]:
retriever = store.as_retriever(search_kwargs={"k":5})

### Create a Prompt

In [14]:
from langchain_core.prompts import ChatPromptTemplate

prompt = ChatPromptTemplate.from_template("""
Answer the question using the given context only.

Context:
{context}

Question:
{input}
""")

### Create a document Chain

In [ ]:
from langchain.chains.combine_documents.stuff import create_stuff_documents_chain

doc_chain = create_stuff_documents_chain(
    model,
    prompt
)

### Create Retrieve Chain

In [2]:
from langchain.chains.retrieval import create_retrieval_chain
retrieval_chain = create_retrieval_chain(
    retriever,
    doc_chain,
)

### Create a Query,

In [4]:
query = retrieval_chain.invoke({
    "input":"What is LangChain"
})

In [15]:
store.similarity_search("what is Agentic AI")

[Document(id='37864ed0-8481-4200-8394-8bad38c5b7ce', metadata={'source': 'https://www.ibm.com/think/topics/langchain', 'title': 'What Is LangChain? | IBM', 'description': 'LangChain is an open source orchestration framework for the development of applications using large language models (LLMs), like chatbots and virtual agents.\u202f', 'language': 'en'}, page_content='Why is agentic AI important?\n\n\n\n\n\n\nAgentic AI vs generative AI\n\n\n\n\n\nCaret right\n\nAI agent development\n\n\n\n\nWhat is AI agent development?\n\n\n\n\nAgentOps\n\n\n\n\nHow to build an AI agent\n\n\n\n\n\n\n\nCaret right\n\nTypes of AI agents\n\n\n\n\nOverview\n\n\n\n\nGoal-based agent\n\n\n\n\nModel-based reflex agent\n\n\n\n\nSimple reflex agent\n\n\n\n\nUtility-based agent\n\n\n\n\n\n\n\n\n\nCaret right\n\nComponents\n\n\n\n\nOverview\n\n\n\n\n\nCaret right\n\nAgentic workflows\n\n\n\n\nWhat are agentic workflows?\n\n\n\n\nTutorial: Agentic workflows\n\n\n\n\n\n\nCommunication'),
 Document(id='c87b5515-81